# Выбор локации для скважины

Мы работаем в добывающей компании «ГлавРосГосНефть». Нам поставили задачу - решить, где бурить новую скважину.

Для решения задачи нам были предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. По этим данным мы построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Далее мы проанализируем возможную прибыль и риски, используя технику *Bootstrap.*

Как мы будем выбирать локацию:

- В избранном регионе найдем месторождения, для каждого определим значения признаков;
- Построим модель и оценим объём запасов;
- Выберем месторождения с самым высокими оценками значений. 

Однако, необходимо также помнить, что:

- Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины. Согласно условиям заказчика, бюджет на разработку скважин в регионе — 10 млрд рублей. Согласно условиям заказчика, доход с каждой единицы продукта составляет 450 тыс. рублей. При этом объём указан в тысячах баррелей.
- Прибыль равна суммарной прибыли отобранных месторождений. 

Соответственно:

- Для обучения модели будем использовать линейную регрессию.
- При разведке региона исследуем 500 точек, из которых с помощью машинного обучения выберем 200 лучших для разработки.
- После оценки рисков оставим лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выберем регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

Импортируем необходимые библиотеки

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import RandomState

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


pd.set_option('display.float_format', '{:,.3f}'.format) 

Импортируем наши датасеты.

In [5]:
dataset_names = ['/datasets/geo_data_0.csv', '/datasets/geo_data_1.csv', '/datasets/geo_data_2.csv']
data_1, data_2, data_3 = [pd.read_csv(name) for name in dataset_names]

Посмотрим на наши данные.

In [6]:
display(data_1.head(), data_2.head(), data_3.head())

,id,f0,f1,f2,product
0,txEyH,0.706,-0.498,1.221,105.280
1,2acmU,1.335,-0.340,4.365,73.038
2,409Wp,1.023,0.152,1.420,85.266
3,iJLyR,-0.032,0.139,2.979,168.621
4,Xdl7t,1.988,0.155,4.752,154.037


,id,f0,f1,f2,product
0,kBEdx,-15.001,-8.276,-0.006,3.179
1,62mP7,14.272,-3.475,0.999,26.953
2,vyE1P,6.263,-5.948,5.001,134.766
3,KcrkZ,-13.081,-11.506,4.999,137.945
4,AHL4O,12.702,-8.147,5.004,134.766


,id,f0,f1,f2,product
0,fwXo0,-1.147,0.963,-0.829,27.759
1,WJtFt,0.263,0.270,-2.530,56.070
2,ovLUW,0.195,0.289,-5.586,62.872
3,q6cA6,2.236,-0.554,0.930,114.573
4,WPMUX,-0.516,1.716,5.899,149.601


Согласно заказчику, столбцы означают следующее:

- `id` — уникальный идентификатор скважины;
- `f0, f1, f2` — три значимых признака точек;
- `product` — объём запасов в скважине (тыс. баррелей).

Соответственно, столбцы `f0, f1 и f2` в нашем случае будут признаками, по которым мы и будем предсказывать наш целевой признак `product`.

Посмотрим на то, как распрелены значения наших признаков. Нас интересует только размах значений, поэтому посмотрим только минимальные и максимальные значения.

In [7]:
data_1.describe().loc[['min', 'max'], ['f0', 'f1', 'f2']]

,f0,f1,f2
min,-1.409,-0.848,-12.088
max,2.362,1.344,16.004


Видно, что наши данные распределены не одинаково. Поэтому, перед обучением, необходимо будет их стандартизировать.

Теперь разделим наши датасеты на тренировочную и валидационную выборки в соотношении 75:25. Для этого воспользуемся функцией `train_test_split` из библиотеки `sklearn`.

In [8]:
data_1_train, data_1_valid, data_2_train, data_2_valid, \
data_3_train, data_3_valid = train_test_split(data_1, data_2, data_3, train_size=0.75, random_state=123456789)

Посмотрим на размеры полученных выборок.

In [9]:
print(f'''Количество строк в исходных датасетах: {data_1.shape[0], data_2.shape[0], data_3.shape[0]}
Количество строк в тренировочной выборке: {data_1_train.shape[0], data_2_train.shape[0], data_3_train.shape[0]}
Количество строк в валидационной выборке: {data_1_valid.shape[0], data_2_valid.shape[0], data_3_valid.shape[0]}''')

Количество строк в исходных датасетах: (100000, 100000, 100000)
Количество строк в тренировочной выборке: (75000, 75000, 75000)
Количество строк в валидационной выборке: (25000, 25000, 25000)


Видно, что данные разбили правильно.

Осталось только разбить выборки на признаки и целевой признак. Для этого также напишем функцию, которая сохраняет в `features` только столбцы `f0, f1 и f2`. При этом в `target` сохраним столбец `product`.

In [10]:
def features_target(*data):
    data_to_return = []
    for dataset in data:
        features = dataset.drop(['id', 'product'], axis=1)
        target = dataset['product']
        data_to_return.extend([features, target])
    return data_to_return

Разделим все выборки.

In [11]:
data_1_train_features, data_1_train_target, data_2_train_features,  data_2_train_target, \
data_3_train_features, data_3_train_target = features_target(data_1_train, data_2_train, data_3_train)

data_1_valid_features, data_1_valid_target, data_2_valid_features,  data_2_valid_target, \
data_3_valid_features, data_3_valid_target = features_target(data_1_valid, data_2_valid, data_3_valid)

Посмотрим на наши выборки.

In [12]:
display(data_1_train_features.head(), data_1_train_target.to_frame().head())
display(data_1_valid_features.head(), data_1_valid_target.to_frame().head())

,f0,f1,f2
88125,-0.266,1.050,0.825
13040,-0.831,0.192,1.922
1866,0.129,1.067,-2.132
15252,0.464,0.834,6.456
38331,0.200,-0.429,1.311


,product
88125,85.142
13040,65.180
1866,14.668
15252,119.791
38331,108.708


,f0,f1,f2
49990,0.355,-0.209,8.634
55988,0.145,-0.206,-0.241
19888,0.989,-0.419,5.781
74059,1.130,-0.474,2.644
97457,1.648,-0.348,7.915


,product
49990,184.511
55988,86.956
19888,136.524
74059,64.873
97457,150.568


Видно, что данные были успешно разбиты.

**Вывод:**
- Данные были импортированы. Они представляют собой идентификатор скважины, некоторые важные признаки скважины и объем её запасов.
- Для обучения данные были разделены на тренировочную b валидационную выборки в соотношении 75:25. 
- Затем все выборки были разбиты на `features`, которые содержат только важные признаки скважины `f0, f1, f2` и `target`, содержащий ключевой признак `product`.

## Обучение и проверка модели

Так как нам необходимо предсказывать количественные значения на основе других количественных значений, построим модель линейной регрессии для скважин в разных регионах. При этом, будем считать RMSE и средний запас предсказанного сырья для каждого региона. 

Сначала напишем функцию, которая возвращает необходимые нам параметры.

In [13]:
def model_fitting(train_features, train_target, valid_features, valid_target):
    model = LinearRegression()
    model.fit(train_features, train_target)
    predictions = model.predict(valid_features)
    rmse = mean_squared_error(valid_target, predictions) ** 0.5
    return predictions.mean(), rmse, predictions

Обучим модели.

In [14]:
data_1_predictions = model_fitting(data_1_train_features, data_1_train_target, data_1_valid_features, data_1_valid_target)
data_2_predictions = model_fitting(data_2_train_features, data_2_train_target, data_2_valid_features, data_2_valid_target)
data_3_predictions = model_fitting(data_3_train_features, data_3_train_target, data_3_valid_features, data_3_valid_target)

In [15]:
print(f'''Регион 1:
Средний объем: {data_1_predictions[0]:.3f} тыс.баррелей, RMSE: {data_1_predictions[1]:.3f}

Регион 2:
Средний объем: {data_2_predictions[0]:.3f} тыс.баррелей, RMSE: {data_2_predictions[1]:.3f}

Регион 3:
Средний объем: {data_3_predictions[0]:.3f} тыс.баррелей, RMSE: {data_3_predictions[1]:.3f}''')

Регион 1:
Средний объем: 92.601 тыс.баррелей, RMSE: 37.840

Регион 2:
Средний объем: 68.748 тыс.баррелей, RMSE: 0.890

Регион 3:
Средний объем: 94.881 тыс.баррелей, RMSE: 40.231


**Вывод:**

На данном этапе были рассчитаны средние объемы добычи для каждого региона, а также корень от среднеквадратической ошибки RMSE. Видно, что для 2 региона это очень малое значение. Это может быть связано с тем, что для данного региона значения объемов добычи удалось предсказать лучше, чем для других.

## Подготовка к расчёту прибыли

На данном этапе рассчитаем прибыль от разработки скважин в изученных регионах. Сначала сохраним все необходимые для расчетов значения в отдельные параметры: бюджет разработки и цена одной тысячи баррелей.

In [16]:
budget = 10*10**9
barrel_price = 450*10**3

Посчитаем, сколько необходимо добыть баррелей, чтобы разработка была безубыточна.

In [17]:
min_product = budget/barrel_price

In [18]:
print(f'Для того, чтобы разработка была безубыточна, в регионе необходимо добыть как минимум {min_product:.3f} тыс. баррелей')

Для того, чтобы разработка была безубыточна, в регионе необходимо добыть как минимум 22222.222 тыс. баррелей


Посчитаем, сколько прибыли принесет 200 скважин в каждом регионе, если бы каждая содержала среднее значение сырья.

In [19]:
print(f'''Регион 1:
Разработка 200 скважин окупила бы только {200*barrel_price*data_1_predictions[0]/budget*100:.2f} % затрат

Регион 2:
Разработка 200 скважин окупила бы только {200*barrel_price*data_2_predictions[0]/budget*100:.2f} % затрат

Регион 3:
Разработка 200 скважин окупила бы только {200*barrel_price*data_3_predictions[0]/budget*100:.2f} % затрат''')

Регион 1:
Разработка 200 скважин окупила бы только 83.34 % затрат

Регион 2:
Разработка 200 скважин окупила бы только 61.87 % затрат

Регион 3:
Разработка 200 скважин окупила бы только 85.39 % затрат


**Вывод:**

Расчёт прибыли показал, что если предположить, что каждая скважина содержит среднее значение сырья для региона, то все они оказались бы убыточными.

## Расчёт прибыли и рисков 

Напишем функцию, которая рассчитывает прибыль по выбранным скважинам и предсказаниям модели. 

In [65]:
def count_rev(df):
    return sum(df.sort_values(by='predictions', ascending=False)['valid'][:200])*barrel_price

Теперь посчитаем прибыль для каждого региона техникой Bootstrap. Для этого также напишем функцию, возвращающую распределение прибыли.

In [77]:
state = RandomState(12345)

def bootstrap(predictions, data_valid_target, bootstrap_len=1000, subsample_size=500):
    state = RandomState(12345)
    df = pd.DataFrame({'predictions': predictions, 'valid':data_valid_target})
    values = []
    for _ in range(bootstrap_len):
        subsample = df.sample(n=subsample_size, replace=True, random_state=state)
        revenue = count_rev(subsample) - budget
        values.append(revenue)
    return pd.Series(values), (pd.Series(values) < 0).mean()

Посчитаем прибыль для каждого региона, оценим среднюю прибыль, 95% доверительный интервал и долю отрицательных прибылей.

In [78]:
data_1_bootstrap, data_1_negative_prob = bootstrap(data_1_predictions[2], data_1_valid_target)
data_2_bootstrap, data_2_negative_prob = bootstrap(data_2_predictions[2], data_2_valid_target)
data_3_bootstrap, data_3_negative_prob = bootstrap(data_3_predictions[2], data_3_valid_target)

In [80]:
print(f'''Регион 1:
Разработка 200 лучших скважин в среднем принесет \
{data_1_bootstrap.mean()/10**6:.2f} млн. рублей.
95% доверительный интервал: ({data_1_bootstrap.quantile(0.025)/10**6:.2f}, \
{data_1_bootstrap.quantile(0.975)/10**6:.2f}) млн. рублей.
При этом процент отрицательных убытков {data_1_negative_prob*100:.2f} %.

Регион 2:
Разработка 200 лучших скважин в среднем принесет \
{data_2_bootstrap.mean()/10**6:.2f} млн. рублей.
95% доверительный интервал: ({data_2_bootstrap.quantile(0.025)/10**6:.2f}, \
{data_2_bootstrap.quantile(0.975)/10**6:.2f}) млн. рублей.
При этом процент отрицательных убытков {data_2_negative_prob*100:.2f} %.

Регион 3:
Разработка 200 лучших скважин в среднем принесет \
{data_3_bootstrap.mean()/10**6:.2f} млн. рублей.
95% доверительный интервал: ({data_3_bootstrap.quantile(0.025)/10**6:.2f}, \
{data_3_bootstrap.quantile(0.975)/10**6:.2f}) млн. рублей.
При этом процент отрицательных убытков {data_3_negative_prob*100:.2f} %.''')

Регион 1:
Разработка 200 лучших скважин в среднем принесет 418.70 млн. рублей.
95% доверительный интервал: (-99.77, 927.56) млн. рублей.
При этом процент отрицательных убытков 5.50 %.

Регион 2:
Разработка 200 лучших скважин в среднем принесет 458.25 млн. рублей.
95% доверительный интервал: (63.10, 875.44) млн. рублей.
При этом процент отрицательных убытков 1.10 %.

Регион 3:
Разработка 200 лучших скважин в среднем принесет 381.66 млн. рублей.
95% доверительный интервал: (-176.11, 892.11) млн. рублей.
При этом процент отрицательных убытков 9.20 %.


**Вывод:**

Техника Boostrap показала, что самым прибыльным является 2 регион со средним значением 458.25 млн. рублей. Также, он является единственным регионом, который попадает под требование заказчика и имеет процент убыточности меньше 2.5 %.

## Общий вывод

В данном проекте была построена модель, которая предсказывает прибыль с разработки скважин в трех регионах.
1. Сначала были предобработаны исходные данные для трех регионов, и разделены на тренировочную и валидационную выборки.
2. Далее были обучены модели, которые предсказывают объем сырья в скважине на основе ее параметров.
3. Была рассчитана прибыль в каждом регионе на основе среднего значения сырья в скважине. Оказалось, что все регионы в этом случае убыточны.
4. Путем использования техники Bootstrap были рассчитаны средние значения прибыли с 200 лучших скважин из выбранных 500, построены доверительные интервалы и рассчитана вероятность отрицательной прибыли. Было определено, что лучшим регионом является второй, так как он единственный попадет под требование о проценте отрицательных исходов и имеет наибольшую среднюю прибыль.